In [5]:
import pandas as pd

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
DATA_DIR = '/content/drive/MyDrive/Galaxy_Morphology/'
TRAIN_OUTPUT_DIR = DATA_DIR + 'label_preprocess/final/'
TEST_OUTPUT_DIR = DATA_DIR + 'unlabel_preprocess/final/'

In [8]:
csv_file_path = DATA_DIR+"training_solutions_rev1.csv"

# Read the CSV file into a Pandas DataFrame
df = pd.read_csv(csv_file_path)

# Display the first few rows of the DataFrame
df

,GalaxyID,Class1.1,Class1.2,Class1.3,Class2.1,Class2.2,Class3.1,Class3.2,Class4.1,Class4.2,...,Class9.3,Class10.1,Class10.2,Class10.3,Class11.1,Class11.2,Class11.3,Class11.4,Class11.5,Class11.6
0,100008,0.383147,0.616853,0.000000,0.000000,0.616853,0.038452,0.578401,0.418398,0.198455,...,0.000000,0.279952,0.138445,0.000000,0.000000,0.092886,0.000000,0.000000,0.0,0.325512
1,100023,0.327001,0.663777,0.009222,0.031178,0.632599,0.467370,0.165229,0.591328,0.041271,...,0.018764,0.000000,0.131378,0.459950,0.000000,0.591328,0.000000,0.000000,0.0,0.000000
2,100053,0.765717,0.177352,0.056931,0.000000,0.177352,0.000000,0.177352,0.000000,0.177352,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,100078,0.693377,0.238564,0.068059,0.000000,0.238564,0.109493,0.129071,0.189098,0.049466,...,0.000000,0.094549,0.000000,0.094549,0.189098,0.000000,0.000000,0.000000,0.0,0.000000
4,100090,0.933839,0.000000,0.066161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61573,999948,0.510379,0.489621,0.000000,0.059207,0.430414,0.000000,0.430414,0.226257,0.204157,...,0.000000,0.226257,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.226257
61574,999950,0.901216,0.098784,0.000000,0.000000,0.098784,0.000000,0.098784,0.000000,0.098784,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
61575,999958,0.202841,0.777376,0.019783,0.116962,0.660414,0.067245,0.593168,0.140022,0.520391,...,0.000000,0.000000,0.090673,0.049349,0.000000,0.067726,0.000000,0.000000,0.0,0.072296
61576,999964,0.091000,0.909000,0.000000,0.045450,0.863550,0.022452,0.841098,0.795330,0.068220,...,0.000000,0.068398,0.318132,0.408799,0.227464,0.408799,0.090668,0.023065,0.0,0.045334


In [9]:
def map_to_five_categories(row):
    # Check the first question (smooth, features/disk, star/artifact)
    if row['Class1.1'] >= row['Class1.2'] and row['Class1.1'] >= row['Class1.3']:
        category = 'round' if row['Class7.1'] >= row['Class7.2'] and row['Class7.1'] >= row['Class7.3'] else 'other'
    elif row['Class1.2'] >= row['Class1.1'] and row['Class1.2'] >= row['Class1.3']:
        category = 'disc' if row['Class7.1'] >= row['Class7.2'] and row['Class7.1'] >= row['Class7.3'] else 'other'
    elif row['Class1.3'] >= row['Class1.1'] and row['Class1.3'] >= row['Class1.2']:
        category = 'spiral' if row['Class11.1'] >= row['Class11.2'] and row['Class11.1'] >= row['Class11.3'] and row['Class11.1'] >= row['Class11.4'] and row['Class11.1'] >= row['Class11.5'] and row['Class11.1'] >= row['Class11.6'] else 'other'
    else:
        category = 'other'

    # Create a new DataFrame with 1 in the assigned category and 0 in others
    new_row = pd.Series({'GalaxyID': row['GalaxyID'],'disc': 1 if category == 'disc' else 0,
                         'spiral': 1 if category == 'spiral' else 0,
                         'elliptical': 0,  # No direct mapping to elliptical in the given decision tree
                         'round': 1 if category == 'round' else 0,
                         'other': 1 if category == 'other' else 0})
    return new_row

In [10]:
# Apply the mapping function to each row in the DataFrame
new_categories_df = df.apply(map_to_five_categories, axis=1)

In [11]:
new_categories_df

,GalaxyID,disc,spiral,elliptical,round,other
0,100008.0,1.0,0.0,0.0,0.0,0.0
1,100023.0,0.0,0.0,0.0,0.0,1.0
2,100053.0,0.0,0.0,0.0,0.0,1.0
3,100078.0,0.0,0.0,0.0,1.0,0.0
4,100090.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...
61573,999948.0,0.0,0.0,0.0,0.0,1.0
61574,999950.0,0.0,0.0,0.0,0.0,1.0
61575,999958.0,0.0,0.0,0.0,0.0,1.0
61576,999964.0,1.0,0.0,0.0,0.0,0.0


In [12]:
new_categories_df.iloc[:, 1:].sum()

disc           8830.0
spiral           39.0
elliptical        0.0
round         12039.0
other         40670.0
dtype: float64

In [13]:
new_categories_df.to_csv(DATA_DIR+'galaxy_5classes.csv', index=False)

In [14]:
# Set the number of GalaxyIDs to randomly select from each category
n = 1000
seed_value = 42  # You can choose any integer for the seed

# Perform random sampling for each category with a specified seed
disc_ids = new_categories_df[new_categories_df['disc'] == 1.0]['GalaxyID'].sample(n, random_state=seed_value)
round_ids = new_categories_df[new_categories_df['round'] == 1.0]['GalaxyID'].sample(n, random_state=seed_value)
other_ids = new_categories_df[new_categories_df['other'] == 1.0]['GalaxyID'].sample(n, random_state=seed_value)

# Concatenate the randomly selected GalaxyIDs
selected_ids = pd.concat([disc_ids, round_ids, other_ids])

# Save the randomly selected GalaxyIDs to a CSV file
#selected_ids.to_csv('randomly_selected_galaxy_ids.csv', index=False)

In [15]:
disc_ids.sample(5)

44703    751809.0
52177    860328.0
50751    839307.0
13510    300561.0
54229    891429.0
Name: GalaxyID, dtype: float64

In [16]:
round_ids.sample(5)

18007    365038.0
23487    443003.0
15516    329588.0
42522    720638.0
5871     186990.0
Name: GalaxyID, dtype: float64

In [17]:
other_ids.sample(5)

5840     186598.0
24324    455121.0
60267    980977.0
25697    475653.0
9254     237065.0
Name: GalaxyID, dtype: float64

In [ ]:
selected_ids=selected_ids.astype(int).astype(str).tolist()
selected_ids

['205103',
 '951202',
 '715106',
 '391334',
 '767960',
 '236075',
 '519208',
 '197611',
 '512270',
 '350662',
 '409126',
 '787498',
 '592595',
 '412394',
 '250298',
 '220284',
 '904007',
 '134880',
 '306468',
 '491671',
 '271943',
 '709218',
 '264210',
 '441888',
 '944173',
 '600574',
 '679757',
 '438841',
 '207631',
 '279863',
 '177639',
 '788286',
 '218825',
 '622691',
 '510726',
 '544041',
 '121768',
 '458210',
 '815197',
 '108601',
 '295420',
 '160261',
 '300561',
 '198375',
 '938804',
 '249466',
 '949393',
 '668979',
 '626169',
 '721750',
 '187757',
 '766631',
 '557011',
 '813751',
 '798244',
 '907080',
 '168057',
 '530256',
 '403715',
 '319381',
 '829058',
 '609976',
 '167074',
 '879775',
 '165688',
 '931523',
 '285733',
 '536465',
 '209789',
 '759689',
 '633716',
 '333063',
 '443800',
 '765815',
 '353780',
 '597091',
 '503890',
 '648468',
 '780415',
 '924932',
 '410206',
 '230809',
 '641448',
 '710333',
 '763191',
 '496780',
 '218701',
 '848488',
 '110063',
 '985152',
 '303144',

In [ ]:
DATA_DIR = '/content/drive/MyDrive/Galaxy_Morphology/'
TRAIN_ZIP_PATH = DATA_DIR + 'train.zip'
TRAIN_OUTPUT_DIR = DATA_DIR + 'label_preprocess/'

In [ ]:
from zipfile import ZipFile

def unzip_files(zip_path, output_dir, selected_ids=None):
    with ZipFile(zip_path, 'r') as zip_ref:
        try:
            # Get the list of files in the zip archive
            file_list = zip_ref.namelist()

            # If selected_ids is specified, filter the file list to match selected IDs
            if selected_ids:
                selected_files = [f"{id}.jpg" for id in selected_ids]
                file_list = [file for file in file_list if any(f in file for f in selected_files)]

            # Extract the selected files to the output directory
            zip_ref.extractall(output_dir, members=file_list)
        except Exception as e:
            print(f"Error occurred during extraction: {e}")

In [ ]:
unzip_files(TRAIN_ZIP_PATH, TRAIN_OUTPUT_DIR, selected_ids=selected_ids)

Our Revised Strategy :

1. The galaxy zoo challenge wanted us to predict the distribution out of 37 classes for each of the images in test using training_solutions_rev1.csv.. meaning it's a regression problem.

## our innovation : in line with https://cs229.stanford.edu/proj2016/report/GauthierJainNoordeh-GalaxyMorphology-report.pdf we converted galaxy zoo to a classification problem where the training galaxies are one of disc, spiral,	elliptical,	round,	other. We took a qualitative classification using the decision tree given Table 2 of https://arxiv.org/pdf/1308.3496.pdf - it's a little different from Hubble classification but should demonstrate feasibility of our automation techniques in galaxy classification.

Disc (disc):

Identified by responses indicating the galaxy is viewed edge-on (Question 2) and has a disk structure (Question 7).
Probability values in Class1.2 are considered for this category.

Spiral (spiral):

Identified by responses indicating the presence of spiral arms (Question 4 and Question 11).
Probability values in Class1.3 and Class11.1 are considered for this category.

Elliptical (elliptical):

Identified by responses indicating the galaxy is smooth and rounded with no signs of features or disk (Question 1).
Probability values in Class1.1 are considered for this category.

Round (round):

Identified by responses indicating the galaxy is simply smooth and rounded (Question 1) or has an odd feature such as a ring or lens (Question 8).
Probability values in Class1.1 and Class7.1 are considered for this category.

Other (other):

Assigned to galaxies that don't fit into the above categories, including those with irregularities, disturbances, or features not captured in the primary categories.
Assigned if none of the above conditions are met in the decision tree.

## Steps for our semi supervised learning:
## Initial Training:

Use your labeled dataset (61578 labeled galaxy images) to train your SVM and KNN models. Divide the labeled data into training and validation sets for model evaluation.

## SVM and KNN Prediction:

Use the trained SVM and KNN models to predict labels for the unlabeled dataset (79975 images). These predictions become pseudo-labels.
## Pseudo-labeling:

Combine the pseudo-labels with the labeled dataset, creating a larger dataset for further training.

## Semi-supervised Learning:

Retrain your SVM and KNN models using the combined dataset.

## K-Means Clustering:

Use K-Means clustering on the unlabeled dataset, treating each cluster as a separate class. Assign the cluster labels to the images in each cluster.
Combining Predictions:

Combine the predictions from SVM, KNN, and K-Means for each image. For example, you could use a majority voting scheme.
Model Evaluation:

Use your validation set to evaluate the performance of your models, and possibly tune hyperparameters.

## Confusion Matrix:

Generate a confusion matrix using the true labels and the combined predictions. Calculate metrics such as accuracy, precision, recall, and F1-score for each class.

Data Split:

You might consider an initial split of the labeled dataset, e.g., 80% for training and 20% for validation.
For the unlabeled dataset, you can use a portion for initial clustering and the rest for semi-supervised learning.
Adjust the ratios based on the size of your datasets and computational resources.

Confusion Matrix:

Create a confusion matrix using the true labels and the final predictions (combined predictions from SVM, KNN, and K-Means).
For a 5-class problem, the confusion matrix would be a 5x5 matrix where each row represents the true class, and each column represents the predicted class.